# Playfair implementation
Julian David Osorio Amaya

## Playfair class

In [76]:
class Playfair:
    def __init__(self):
        pass

    @staticmethod
    def generate_key_table(key):
        key = key.lower()
        seen = set()
        key_chars = []

        # Add unique characters from key
        for k in key:
            if k.isalpha() and k not in seen:
                if k == 'j':
                    k = 'i'
                if k not in seen:
                    seen.add(k)
                    key_chars.append(k)

        # Add remaining alphabet characters
        for i in range(26):
            char = chr(97 + i)
            if char == 'j':
                continue
            if char not in seen:
                seen.add(char)
                key_chars.append(char)

        # Build 5x5 table
        enc_table = []
        for i in range(5):
            row = key_chars[i*5:(i+1)*5]
            enc_table.append(row)

        return enc_table

    def find_position(self, char, table):
        """Find row and column of character in table"""
        if char == 'j':
            char = 'i'
        for i, row in enumerate(table):
            for j, c in enumerate(row):
                if c == char:
                    return i, j
        return None, None

    def prepare_message(self, message):
        """Prepare message for encryption - create digraphs"""
        message = message.lower().replace(' ', '')
        message = message.replace('j', 'i')

        pairs = []
        i = 0
        while i < len(message):
            a = message[i]

            if i + 1 < len(message):
                b = message[i + 1]
                if a == b:
                    # Insert 'x' between duplicates
                    pairs.append(a + 'x')
                    i += 1
                else:
                    pairs.append(a + b)
                    i += 2
            else:
                # Odd length, pad with 'x'
                pairs.append(a + 'x')
                i += 1

        return pairs

    def encrypt_pair(self, pair, table):
        """Encrypt a single pair of characters"""
        r1, c1 = self.find_position(pair[0], table)
        r2, c2 = self.find_position(pair[1], table)

        if r1 == r2:
            # Same row - shift right
            return table[r1][(c1 + 1) % 5] + table[r2][(c2 + 1) % 5]
        elif c1 == c2:
            # Same column - shift down
            return table[(r1 + 1) % 5][c1] + table[(r2 + 1) % 5][c2]
        else:
            # Rectangle - swap columns
            return table[r1][c2] + table[r2][c1]

    def decrypt_pair(self, pair, table):
        """Decrypt a single pair of characters"""
        r1, c1 = self.find_position(pair[0], table)
        r2, c2 = self.find_position(pair[1], table)

        if r1 == r2:
            # Same row - shift left
            return table[r1][(c1 - 1) % 5] + table[r2][(c2 - 1) % 5]
        elif c1 == c2:
            # Same column - shift up
            return table[(r1 - 1) % 5][c1] + table[(r2 - 1) % 5][c2]
        else:
            # Rectangle - swap columns (same as encryption)
            return table[r1][c2] + table[r2][c1]

    def encrypt(self, message, key):
        enc_table = self.generate_key_table(key)
        print(f"The generated encryption table is: {enc_table}")
        pairs = self.prepare_message(message)

        encrypted = ''
        for pair in pairs:
            encrypted += self.encrypt_pair(pair, enc_table)

        return encrypted

    def decrypt(self, message, key):
        dec_table = self.generate_key_table(key)
        print(f"The generated decryption table is: {dec_table}")

        # Split ciphertext into pairs
        pairs = [message[i:i+2] for i in range(0, len(message), 2)]

        decrypted = ''
        for pair in pairs:
            decrypted += self.decrypt_pair(pair, dec_table)

        return decrypted

    def compute(self, method, message, key):
      if method == 0:
        return self.encrypt(message, key)
      elif method == 1:
        return self.decrypt(message, key)



## Code execution

In [79]:
playfair = Playfair()
key = "key"
plaintext = "It doesnt even matter how hard you try"

encrypted = playfair.compute(0, plaintext, key)
print(f"Encrypted: {encrypted}")

decrypted = playfair.compute(1, encrypted, key)
print(f"Decrypted: {decrypted}")

The generated encryption table is: [['k', 'e', 'y', 'a', 'b'], ['c', 'd', 'f', 'g', 'h'], ['i', 'l', 'm', 'n', 'o'], ['p', 'q', 'r', 's', 't'], ['u', 'v', 'w', 'x', 'z']]
Encrypted: ophlaqosdealnyszqbtfmzgbqfbmzpwf
The generated decryption table is: [['k', 'e', 'y', 'a', 'b'], ['c', 'd', 'f', 'g', 'h'], ['i', 'l', 'm', 'n', 'o'], ['p', 'q', 'r', 's', 't'], ['u', 'v', 'w', 'x', 'z']]
Decrypted: itdoesntevenmatxterhowhardyoutry
